In [1]:
#Information selection
import numpy as np
import pandas as pd
from sklearn.neighbors import KernelDensity
from scipy import stats
from matplotlib import pyplot as plt
from scipy.stats import gaussian_kde
from scipy.integrate import dblquad,quad  
from sklearn.model_selection import GridSearchCV
from scipy.integrate import quad 
import warnings
warnings.filterwarnings("ignore")

# 读取数据集
df=pd.read_csv('PRODIGY dataset.csv')
column_indexes =np.r_[1,4:13]
data=df.iloc[:,column_indexes]
print(data)

feature=np.array(data.columns)[1:]
Y= data['Binding_affinity'].values

xi_dict = {}
for i, feature_name in enumerate(feature): 
    xi_dict[f'X{i+1}'] = data[feature_name].values

data_values = {}
for i in range(1, 10):
    data_values[f'data{i}'] = np.array([xi_dict[f'X{i}'], Y])

    
#定义熵的计算公式：
def compute_entropy_from_pdf(pdf_func, mean, std, lower=None, upper=None):
    if lower is None:
        lower = mean - 5 * std
    if upper is None:
        upper = mean + 5 * std

    def integrand(x):
        eps = 1e-10  
        try:
            p = pdf_func(x)
        except:
            return 0.0
        p = max(p, eps)
        return -p * np.log(p)

    entropy_estimate, _ = quad(integrand, lower, upper)
    return entropy_estimate

m=4  
n0=len(Y)  
values0 = data['Binding_affinity'].values.reshape(-1, 1)
min_std0=min(np.std(data['Binding_affinity']),((data['Binding_affinity'].quantile(0.75)-data['Binding_affinity'].quantile(0.25))/1.34))
better_bandwidth0=1.06 * min_std0 * len(Y) ** (-1 / 5) 
bandwidths0 = np.linspace(0.1 * better_bandwidth0, 2 * better_bandwidth0, 1000)
grid0 = GridSearchCV(KernelDensity(kernel='gaussian'),
                    {'bandwidth': bandwidths0},
                    cv=m)  
grid0.fit(values0)
best_bandwidth0 = grid0.best_estimator_.bandwidth
kde0 = KernelDensity(kernel='gaussian', bandwidth=best_bandwidth0)
kde0.fit(values0)

def pdf(y):
    log_prob = kde0.score_samples([[y]])  
    return np.exp(log_prob)

#计算 H(Y)
HY = compute_entropy_from_pdf(pdf, np.mean(Y), np.std(Y))

MI_withy_dict={}
NMI_max_dict = {} 

#计算各特征xi与y的互信息和最大归一化互信息
for i in range(1, 10):
    covariance_matrix = np.cov(data_values[f'data{i}'])
    std_devs = np.sqrt(np.diag(covariance_matrix))
    n = data_values[f'data{i}'].shape[1]
    d = data_values[f'data{i}'].shape[0]
    bandwidths = ((4 / (d+2)) ** (1 / (d+4)) * n ** (-1 / (d+4))) * std_devs 

    def custom_bandwidth(*args):
        return np.diag(bandwidths)

    kde_xy = gaussian_kde(data_values[f'data{i}'], bw_method=custom_bandwidth)

    def integrand(y,x):
        return kde_xy([x,y]) 
    
    valuesi = xi_dict[f'X{i}'].reshape(-1, 1) 
    min_stdi=min(np.std(data[feature[i-1]]),((data[feature[i-1]].quantile(0.75)-data[feature[i-1]].quantile(0.25))/1.34))
    better_bandwidthi=1.06 * min_stdi * n ** (-1 / 5) 
    bandwidthsi = np.linspace(0.1 * better_bandwidthi, 2 * better_bandwidthi, 1000)
    gridi = GridSearchCV(KernelDensity(kernel='gaussian'),
                        {'bandwidth': bandwidthsi},
                        cv=m)  
    gridi.fit(valuesi)
    best_bandwidthi = gridi.best_estimator_.bandwidth
    kdei = KernelDensity(kernel='gaussian', bandwidth=best_bandwidthi)
    kdei.fit(valuesi)

    def pdfi(x):
        log_prob = kdei.score_samples([[x]]) 
        return np.exp(log_prob)

    def mutual_information_integrandi(y, x):
        eps = 1e-10  
        pxy = integrand(y, x)
        px = pdfi(x)
        py = pdf(y)

        if pxy > eps and px > eps and py > eps:
            return pxy * np.log((pxy + eps) / ((px + eps) * (py + eps)))
        else:
            return 0

    x_range = np.std(xi_dict[f'X{i}']) * 5
    y_range = np.std(Y) * 5

    mi, mi_error = dblquad(
        mutual_information_integrandi, 
        np.mean(xi_dict[f'X{i}']) - x_range,   
        np.mean(xi_dict[f'X{i}']) + x_range, 
        lambda x: np.mean(Y) - y_range, 
        lambda x: np.mean(Y) + y_range, 
    )

    MI_withy_dict[feature[i-1]] = mi
    
    HX = compute_entropy_from_pdf(pdfi, np.mean(xi_dict[f'X{i}']), np.std(xi_dict[f'X{i}']))
    nmi_max = mi / max(HX,HY)
    NMI_max_dict[feature[i-1]] = nmi_max
    
#输出所有特征的互信息与NMI
print("\n=== 各特征与∆G的互信息 MI ===")
for k, v in MI_withy_dict.items():
    print(f"{k}: {v:.4f}")
print("\n=== 各特征与∆G的最大归一化互信息 NMI ===")
for k, v in NMI_max_dict.items():
    print(f"{k}: {v:.4f}")

    Binding_affinity  Ics_charg-charg  Ics_charg-polar  Ics_charg-apolar  \
0                9.3                5                4                20   
1               13.1                3                2                19   
2                6.4                1                7                14   
3                5.3                8               20                 7   
4               12.1                9                8                20   
..               ...              ...              ...               ...   
76              10.7                8               14                12   
77               9.6                4                7                 6   
78               8.8                6                7                14   
79              14.5                3                5                14   
80              11.3                0                3                16   

    Ics_polar-polar  Ics_polar-apolar  Ics_apolar-apolar  %NISapol  %NISpol  \
0       